In [1]:
import tensorflow as tf
import numpy as np
import os
import time


from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

# Warum auch immer, bekomme ich einen fehler wenn ich das hier nicht drinne habe. 
config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.9
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)


path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [2]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


In [3]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [4]:
vocab = sorted(set(text))
print(f'Einzigartige Chars: {len(vocab)}')

Einzigartige Chars: 65


In [5]:
# Zwei Lookup tables, chars -> zahlen & zahlen -> chars
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text]) 

In [6]:
for char, _ in zip(char2idx, range(20)):
    print(f'{repr(char)}: {char2idx[char]}')

'\n': 0
' ': 1
'!': 2
'$': 3
'&': 4
"'": 5
',': 6
'-': 7
'.': 8
'3': 9
':': 10
';': 11
'?': 12
'A': 13
'B': 14
'C': 15
'D': 16
'E': 17
'F': 18
'G': 19


In [7]:
# Wie ist der Text mapped?
print(f'{repr(text[:13])} ------ mapped ------ > {text_as_int[:13]}')

'First Citizen' ------ mapped ------ > [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [8]:
# Das Trainieren ist relativ einfach
# Wir versuchen ein vorhersage Modell auf Char basis. Wir haben bereits einen (langen) original Text. 
# Wir gehen einfach für jeden char durch das Modell und können testen ob es richtig vorhersagt, da wir den Text haben
# RNN haben den Vorteil, einen internen Zustand zu haben, der das vorherige Ergebniss speichert. Dadurch gut für Wörter / Sätze geeignet

# Wir trainieren das Modell in Sequenzen. Bsp. Seq_leng = 4 und der Text = "Hello" input = "Hell" output = "ello"

seq_length = 100
examples_per_epoch = len(text)//(seq_length + 1)
print(examples_per_epoch)

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

11043
F
i
r
s
t


In [9]:
sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [10]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [11]:
# Example copy pasta wie das Modell lernen würde:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 18 ('F')
  expected output: 47 ('i')
Step    1
  input: 47 ('i')
  expected output: 56 ('r')
Step    2
  input: 56 ('r')
  expected output: 57 ('s')
Step    3
  input: 57 ('s')
  expected output: 58 ('t')
Step    4
  input: 58 ('t')
  expected output: 1 (' ')


In [12]:
BATCH_SIZE = 64

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int32, tf.int32)>

In [13]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

In [14]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units, return_sequences=True, stateful=True, recurrent_initializer="glorot_uniform"),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

model = build_model(vocab_size = len(vocab), embedding_dim=embedding_dim, rnn_units=rnn_units, batch_size=BATCH_SIZE)

In [15]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [17]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
sampled_indices

array([59, 13, 12, 56,  5, 55, 60, 63, 50, 46, 45, 15,  4, 46, 15, 19, 20,
       50, 55, 13, 52, 42, 29,  0, 45, 45, 64, 42, 44,  3, 34,  5, 46, 45,
       62, 24, 24,  9, 58, 14, 51, 59, 59, 62, 47, 11, 36, 23, 42, 20, 47,
       64, 61,  9, 52, 56, 21, 20, 11, 58, 17, 36, 52, 51, 25, 24, 33, 11,
       26,  6, 27,  6, 37,  7, 18,  1,  0, 55, 31, 17,  8, 37, 64,  5, 63,
       12, 10, 24, 10, 21, 61, 12, 45, 26, 42, 32,  0, 16, 16, 14],
      dtype=int64)

In [18]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'urselves to do it, but it is a\npower that we have no power to do; for if he show us\nhis wounds and t'

Next Char Predictions: 
 "uA?r'qvylhgC&hCGHlqAndQ\nggzdf$V'hgxLL3tBmuuxi;XKdHizw3nrIH;tEXnmMLU;N,O,Y-F \nqSE.Yz'y?:L:Iw?gNdT\nDDB"


In [19]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print(example_batch_predictions.shape)
print(example_batch_loss.numpy().mean())

(64, 100, 65)
4.174007


In [20]:
model.compile(optimizer="adam", loss=loss)

In [21]:
# Da das Training vermutlich sehr lange dauert, wollen wir Checkpoints haben.
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix, save_weights_only=True)

EPOCHS=10
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
172/172 [==============================] - 9s 51ms/step - loss: 2.6886
Epoch 2/10
172/172 [==============================] - 9s 51ms/step - loss: 1.9632
Epoch 3/10
172/172 [==============================] - 9s 52ms/step - loss: 1.6964
Epoch 4/10
172/172 [==============================] - 9s 50ms/step - loss: 1.5465
Epoch 5/10
172/172 [==============================] - 9s 50ms/step - loss: 1.4576
Epoch 6/10
172/172 [==============================] - 10s 60ms/step - loss: 1.3960
Epoch 7/10
172/172 [==============================] - 10s 59ms/step - loss: 1.3503
Epoch 8/10
172/172 [==============================] - 9s 52ms/step - loss: 1.3115: 0s - lo
Epoch 9/10
172/172 [==============================] - 9s 51ms/step - loss: 1.2765
Epoch 10/10
172/172 [==============================] - 9s 54ms/step - loss: 1.2426


In [22]:
# Um die Batch Size zu ändern muss dass Modell neu erzeugt werden, mit den Weights aus dem Training
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 65)             66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [29]:
def generate_text(model, start_string):
    num_generate = 1000
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    
    text_generated = []
    
    temperature = 0.3 # Tief = mehr vorhersehbar, Hoch = mehr unerwartet
    
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()
        
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])
        
    return (start_string + ''.join(text_generated))

In [30]:
print(generate_text(model, start_string=u"ROMEO: "))

ROMEO: I will not say he lives me to the gate.

GLOUCESTER:
Why, what my son God he be made me so?

PETRUCHIO:
I cannot think the truth of his most country's good and all the third that I was but a man
Than the field of his master and the wars:
The king with the earth and false from his master's peace,
That I may not be so for our foes,
When I may be so far from the house of York.

GLOUCESTER:
The most shall be so fit than the bosom of the death.

Second Servant:
When he shall be the strong and stands in the house of York.

LEONTES:
What may come to me, I say the town,
When I should the peace of the people, when he shall have the deed.

Second Lord:
So shall the common person made thee this man shall not be prone.
The deeds to be in me.

PETRUCHIO:
The gods have we heard the body of the field.

KING HENRY VI:
Ay, sir, then the common worldly then lies of the field.

DUKE VINCENTIO:
The very breath of himself will be so soon as I do not show thee hence to command thee to the sun,
Shall 